In [1]:
# If data exported in gs://aniftos-example-dataset/Test_AutoML_Exports
# read_location = "gs://aniftos-example-dataset/automl_export_example/export_data-test_import_entity-2020-04-09T13:39:13.950Z/text_extraction.csv"
# write_location = "gs://aniftos-example-dataset/automl_export_example/bla/"

read_location = "gs://bonibon/text/export_data-entity_extraction-2020-07-27T16:31:13.349Z/text_extraction.csv"
write_location = "gs://bonibon/text/bla/"
            
read_location= read_location.split('/')
read_bucket_name = read_location[2]
read_object = '/'.join(read_location[3:])

write_location = write_location.split('/')
write_bucket_name = write_location[2]
write_prefix = '/'.join(write_location[3:])

In [2]:
from google.cloud import storage
import sys, os
from io import StringIO
import csv
import time

storage_client = storage.Client()
bucket = storage_client.get_bucket(read_bucket_name)


In [6]:
blob = bucket.get_blob(read_object)
csv_str = blob.download_as_string()
f = StringIO(csv_str.decode())
reader = csv.reader(f, delimiter=',')


seconds = time.time()

write_bucket = storage_client.get_bucket(write_bucket_name)
csv_tmp = ''

# maximum size limit for the output files after the split.
size_limit = 200000000

for row in reader:
    print('\t'.join(row))
    counter=0
    set_type = row[0]
    path = row[1].split('/')
    
    bucket_tmp = storage_client.get_bucket(path[2])
    blob_tmp = bucket.get_blob('/'.join(path[3:]))
    json_data = blob_tmp.download_as_string()
    tmp_str=""
    
    for line in json_data.splitlines():
        if(sys.getsizeof(tmp_str+line.decode("utf-8")+"\n")> size_limit):
            
            filepath = os.path.join(write_prefix,blob_tmp.name.split("/")[-1][:-6]+"_"+str(counter)+".jsonl")
            write_blob = write_bucket.blob(filepath)
            write_blob.upload_from_string(tmp_str) 
            csv_tmp += ",".join([set_type, os.path.join(write_bucket_name,filepath)])+"\n"
            tmp_str = ''
            counter+=1
            
        tmp_str += line.decode("utf-8")+"\n"
     
    filepath = os.path.join(write_prefix,blob_tmp.name.split("/")[-1][:-6]+"_"+str(counter)+".jsonl")
    write_blob = write_bucket.blob(filepath)
    write_blob.upload_from_string(tmp_str)   
    csv_tmp += ",".join([set_type, os.path.join(write_bucket_name,filepath)])+"\n" 
    
write_blob = write_bucket.blob(os.path.join(write_prefix,"output_csv_"+str(seconds)+".csv"))
write_blob.upload_from_string(csv_tmp)   